In [3]:
%reload_ext autoreload
%autoreload 2

In [3]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import UniSkip
from config import *
from datetime import datetime, timedelta

In [4]:
d = DataLoader("./data/all.txt")

Loading text file at ./data/all.txt
Making dictionary for these words
Using cached dictionary at ./data/all.txt.pkl
Making reverse dictionary


In [5]:
mod = UniSkip()
print(USE_CUDA)

False


In [6]:
lr = 3e-4
optimizer = torch.optim.Adam(params=mod.parameters(), lr=lr)

In [7]:
loss_trail = []
last_best_loss = None
current_time = datetime.utcnow()

def debug(i, loss, prev, nex, prev_pred, next_pred):
    global loss_trail
    global last_best_loss
    global current_time

    this_loss = loss.item()
    loss_trail.append(this_loss)
    loss_trail = loss_trail[-20:]
    new_current_time = datetime.utcnow()
    time_elapsed = str(new_current_time - current_time)
    current_time = new_current_time
    print("Iteration {}: time = {} last_best_loss = {}, this_loss = {}".format(
              i, time_elapsed, last_best_loss, this_loss))
    
    print("prev = {}\nnext = {}\npred_prev = {}\npred_next = {}".format(
        d.convert_indices_to_sentences(prev),
        d.convert_indices_to_sentences(nex),
        d.convert_indices_to_sentences(prev_pred),
        d.convert_indices_to_sentences(next_pred),
    ))
    
    try:
        trail_loss = sum(loss_trail)/len(loss_trail)
        if last_best_loss is None or last_best_loss > trail_loss:
            print("Loss improved from {} to {}".format(last_best_loss, trail_loss))
            
            save_loc = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
            save_loc_encoder = "./saved_models/encoder-best".format(lr, VOCAB_SIZE)
            print("saving model at {}".format(save_loc)) 
            torch.save(mod.state_dict(), save_loc) 
            torch.save(mod.encoder.state_dict(), save_loc_encoder) 

            last_best_loss = trail_loss
    except Exception as e:
       print("Couldn't save model because {}".format(e))

In [30]:
print("Starting training...")

# a million iterations
for i in range(0, 10000):
    sentences, lengths = d.fetch_batch(200)

    loss, prev, nex, prev_pred, next_pred  = mod(sentences, lengths)
   

    if i % 10 == 0:
        print(i)
        debug(i, loss, prev, nex, prev_pred, next_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Starting training...
0
Iteration 0: time = 4:23:12.465433 last_best_loss = 15.418782997131348, this_loss = 16.092342376708984
prev = Reaching down, he grabbed her arm and pulled her up and out of the reach of the geese. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = The geese kept following. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = She to he said, her arm. and down her into to down of the room. of the car. of of from to to and to and and and and and and and and and and and and and and and
pred_next = She driver kept following. out from from from and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and


KeyboardInterrupt: 

In [9]:
from scipy import spatial
save_loc_encoder = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
# mod = TheModelClass(*args, **kwargs)
mod.load_state_dict(torch.load(save_loc_encoder))

# sentences, lengths = d.fetch_batch(200)
# output, _= mod.encoder.forward(sentences)
output=[]
sentencetotal=[]
#print(output.shape)
for i in range(0, 30):
    sentences, lengths = d.fetch_batch(200)
    #print(sentences[0].shape)
    outputnow, _= mod.encoder.forward(sentences)
    print(i)
    print(outputnow.shape)
    sentencetotal.append(sentences)
    output.append(outputnow)#output is a list of 2Dtensors here
#print(sentencetotal[1].shape)
print(output)
# print(len(output))

/home/aditi/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


0
torch.Size([200, 1200])
1
torch.Size([200, 1200])
2
torch.Size([200, 1200])
3
torch.Size([200, 1200])
4
torch.Size([200, 1200])
5
torch.Size([200, 1200])
6
torch.Size([200, 1200])
7
torch.Size([200, 1200])
8
torch.Size([200, 1200])
9
torch.Size([200, 1200])
10
torch.Size([200, 1200])
11
torch.Size([200, 1200])
12
torch.Size([200, 1200])
13
torch.Size([200, 1200])
14
torch.Size([200, 1200])
15
torch.Size([200, 1200])
16
torch.Size([200, 1200])
17
torch.Size([200, 1200])
18
torch.Size([200, 1200])
19
torch.Size([200, 1200])
20
torch.Size([200, 1200])
21
torch.Size([200, 1200])
22
torch.Size([200, 1200])
23
torch.Size([200, 1200])
24
torch.Size([200, 1200])
25
torch.Size([200, 1200])
26
torch.Size([200, 1200])
27
torch.Size([200, 1200])
28
torch.Size([200, 1200])
29
torch.Size([200, 1200])
[tensor([[-0.6626,  0.2374,  0.6261,  ...,  0.3751,  0.3939, -0.6425],
        [-0.2512,  0.0597, -0.3330,  ..., -0.9362,  0.5731, -0.5963],
        [-0.4618,  0.3530, -0.1260,  ...,  0.4963,  0.8653,

In [10]:
import numpy as np
#print(sentences.shape)
string = ["he ran his hand inside his coat , double-checking that the unopened letter was still there .."
,         "im sure youll have a glamorous evening , she said , giving an exaggerated wink"
,         "I was trying to decide if it was unseemly to drink tea alone with a strange gentleman in a salubrious hotel, and if I cared about that sort of thing anymore, when Eddie took advantage of my silence."
,         "Savannah flipped her blonde hair and physically stepped between Claire and Brody, her nose turned up in challenge."
,         "She had done it without thinking, even though she wasn't very strong, and now she gritted her teeth as she slammed into the railing, which caused her to cry out in pain."
,         "He sounded bored, but that could have been a result of his accent."
,         "I ought to have been frightened witless, but their light-hearted banter quelled my fear"
,         "The brown eyes briefly flared and a small smile touched his lips."
,         "He wore a well-tailored black suit, untroubled by his impressive frame, a silk hat and gray silk tie."
,         "Im sure well be fine. She bit her lip, and I knew she was trying hard not to smile."]
for j in range(10):
    print(j)
    query= d.convert_sentence_to_indices(string[j])
    #print(query.shape)
    query,_= mod.encoder.forward(query.reshape(1,MAXLEN))
    #print(sentence.shape)
    #print(query[0].shape)
    cosine=[]
    newsen=[]
    for batch in output:
        for sentence in batch:
            #print(sentence.shape)
            newsen.append(sentence)
            cosine.append(torch.nn.functional.cosine_similarity(query[0],sentence,dim=0))
    
    #print(len(cosine))
    
    #i=max(cosine)
    i=cosine.index(max(cosine))
    #print(i)
    #print(newsen[1].tolist())
    #print(len(newsen))
    actual=sentencetotal[i//200]
    
    print(d.convert_indices_to_sentences(actual[i%200]))

0
he asked as the two constables lifted the dazed Dorchester between them. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
1
UNK Brammer UNK in to UNK the rising tension. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
2
I was trying to say I wish I hadnt gotten mad at him so much those last few days. She stood clutching the dog, breathing slowly. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
3
Nan rolled her eyes and held up two fingers. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
4
She had burned her bridges, and burned them badly. EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
5
He rose and greeted me with a dashing smile that I couldn't help but return, despite my UNK stomach. EOS